# 610 WF6300 - Test contra Julio

Replica de WF6300 (mejor Kaggle 4.587M) testeando contra 202107
para comparar ganancia local vs ganancia Kaggle

**Hiperparametros WF6300:**
- num_leaves = 175
- min_data_in_leaf = 304
- num_iterations = 528
- feature_fraction = 0.5
- learning_rate = 0.03

**Semilla:** 153929

### 6.1 Objetivo

Presentar un workflow/pipeline completo al que los estudiantes deberán enriquecer

#### 6.2  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab
*   Bajar el **dataset_historico** al Google Drive y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets


webfiles="https://storage.googleapis.com/open-courses/austral2025-af91/"
destino_local="/content/datasets"
destino_bucket="/content/buckets/b1/datasets"


archivo="dataset_pequeno.csv"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi

#-------

archivo="gerencial_competencia_2025.csv.gz"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi


## 6.3  Workflow

## Inicializacion

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

In [ ]:
require("data.table")

if( !require("R.utils")) install.packages("R.utils")
require("R.utils")

#### Parametros
Si es gerente, no cambie nada
<br>Si es Analista, cambie el nombre del dataset

In [ ]:
# Parámetros - Usando los mismos valores que WF6300
PARAM <- list()
PARAM$semilla_primigenia <- 153929  # Misma semilla que WF6300

PARAM$experimento <- 6301  # Nuevo número para este test local
PARAM$dataset <- "gerencial_competencia_2025.csv.gz"

# Mejores hiperparámetros de WF6300 (ya encontrados por BO - NO corremos BO de nuevo)
PARAM$lgbm$mejores_hiperparametros <- list(
  num_leaves = 175,
  min_data_in_leaf = 304,
  num_iterations = 528
)

cat("Usando hiperparámetros WF6300:\n")
cat("  num_leaves:", 175, "\n")
cat("  min_data_in_leaf:", 304, "\n")
cat("  num_iterations:", 528, "\n")

#### Carpeta del Experimento

In [ ]:
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("WF", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

### 6.3.1   Preprocesamiento del dataset

#### 6.3.1.1  DT incorporar dataset

In [ ]:
# lectura del dataset
dataset <- fread(paste0("/content/datasets/", PARAM$dataset))

#### 6.3.1.2  CA  Catastrophe Analysis
Se intentan reparar las variables que para un mes están con todos los valores en cero.

El método que se utiliza es **Machine Learning** se asigna NA also valores, si ha leido bien, es la "anti imputación de valores faltantes"
<br> Usted podrá aplicar aquí otros métodos

In [ ]:
dataset[ foto_mes==202006, internet:=NA]
dataset[ foto_mes==202006, mrentabilidad:=NA]
dataset[ foto_mes==202006, mrentabilidad_annual:=NA]
dataset[ foto_mes==202006, mcomisiones:=NA]
dataset[ foto_mes==202006, mactivos_margen:=NA]
dataset[ foto_mes==202006, mpasivos_margen:=NA]
dataset[ foto_mes==202006, mcuentas_saldo:=NA]
dataset[ foto_mes==202006, ctarjeta_visa_transacciones:=NA]
dataset[ foto_mes==202006, mtarjeta_visa_consumo:=NA]
dataset[ foto_mes==202006, mtarjeta_master_consumo:=NA]
dataset[ foto_mes==202006, ccallcenter_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]

#### 6.3.1.3  DR  Data Drifting
Se intenta corregir el data drifting, quizas ajustando por IPC ...
<br>Esta parte podrá ser abordada por todos los Analistas y también la Gerenciapero se decide pedagogicamente no incluirla en esta primer version para reducir la carga cognitiva

In [ ]:
# sin codigo en esta primera version del workflow

#### 6.3.1.3  FE_intra_manual Feature Engineering intra-mes

Agrego campos nuevos dentro del mismo mes, SIN considerar la historia.

In [ ]:
# esta funcion atributos presentes existe debido a que las modalidades poseen datasets con distinta cantidad de campos
atributos_presentes <- function( patributos )
{
  atributos <- unique( patributos )
  comun <- intersect( atributos, colnames(dataset) )

  return(  length( atributos ) == length( comun ) )
}

# el mes 1,2, ..12
if( atributos_presentes( c("foto_mes") ))
  dataset[, kmes := foto_mes %% 100]

# variable extraida de una tesis de maestria de Irlanda
if( atributos_presentes( c("mpayroll", "cliente_edad") ))
  dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]


In [ ]:
# visualizo las columas del dataset a esta etapa
colnames(dataset)

#### 6.3.1.4  FE_rf Feature Engineering de nuevas variables a partir de hojas de Random Forest

Esto se mostrará unicamente a la *modalidad Analista Sr*

In [ ]:
# No se implementa Feature Engineering a partir de Random Forest

#### 6.3.1.5  FEhist Feature Engineering historico

El Fature Engineering Histórico es la etapa que más aporta a la ganancia final, ya que enriquece cada registro del dataset con su historia.

Para cada campo del dataset original (*)
se crean lo siguientes campos de a partir de la historia
* lag1  lags de orden 1
* delta1  =  valor actual - lag1
* lag2  lags de orden 2
* delta2  = valor actual - lag2


(*) Excepto para los campos  <numero_de_cliente,  foto_mes,  clase_ternaria>

In [ ]:
# Feature Engineering Historico

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes", "clase_ternaria")
) )

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
    paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# lags de orden 2
dataset[,
    paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
    dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
    dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}


Verificacion de los campos recien creados

In [ ]:
ncol(dataset)
colnames(dataset)

#### 6.3.1.6  FEhist Reduccion dimensionalidad con canaritos

Esta etapa solo se mostrará a la *modalidad Anlista Sr* por algun canal secreto de forma de no confundir a los *Analista Jr*  nni distraer con detalles operativos a la estratégica *Modalidad Gerencial*

In [ ]:
# No se implementa la reduccion de la dimensionalidad con canaritos

### 6.3.2 Modelado

#### 6.3.2.1 Training Strategy

Esta etapa de Workflow de  Training Strategy esta pensada para la *Modalidad Gerencial* que posee el dataset de [202005, 202109]
<br> Si usted es un Analista, posee el periodo de [201901, 202109] y deberá experimentar en que meses le conviene experimentar

<br> A la *Modalidad Gerencial* no se le complicada la vida con el undersampling de los continua, por eso PARAM$trainingstrategy$training_pct <- 1.0
<br> Sin embargo, si usted es  *Analista SR* posee un dataset 50 veces ( filas x columnas) más grande que la *Modalidad Gerencial*  y por un tema de velocidad y experimentación más rápida puede llegar a necesitar activar el undersampling de la clase mayoritaria, a pesar de estar corriendo en Google Cloud.

Se hace una estrategia de entrenamiento muy sencilla, tomando todos los meses posibles, SIN eliminar nada x pandemia ni por ningun otro motivo

* future = 202107  obviamente completo

* final_train =  [ 202005, 202105 ]  SIN undersampling

* training
   * testing = NO HAY
   * validation =  202105   completo, sin undersampling
   * training = [ 202005, 202104 ]  donde se consideran el 100% de los CONTINUA

In [ ]:
# Training Strategy - Misma que WF6300
# Validación contra 202107 para calcular ganancia local
PARAM$trainingstrategy$validate <- c(202107)

# Training: 202005 a 202106 (igual que WF6300)
PARAM$trainingstrategy$training <- c(
  202106, 202105, 202104, 202103, 202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007,
  202006, 202005
)

PARAM$trainingstrategy$training_pct <- 1.0

PARAM$trainingstrategy$positivos <- c("BAJA+1", "BAJA+2")

cat("Training Strategy WF6300:\n")
cat("  Training meses:", length(PARAM$trainingstrategy$training), "\n")
cat("  Validation:", PARAM$trainingstrategy$validate, "\n")

In [ ]:
# seteo la clase01   1={BAJA+1, BAJA+2}   0={CONTINUA}
dataset[, clase01 := ifelse( clase_ternaria %in% PARAM$trainingstrategy$positivos, 1, 0 )]

In [ ]:
# los campos en los que se entrena
campos_buenos <- copy( setdiff(
    colnames(dataset), c("clase_ternaria","clase01","azar"))
)

In [ ]:
# preparo para que se puede hacer undersampling de los CONTINUA
#  solamente por un tema de VELOCIDAD
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset[, azar:=runif(nrow(dataset))]

# undersampling de los CONTINUA
dataset[, fold_train :=  foto_mes %in%  PARAM$trainingstrategy$training &
    (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
     azar < PARAM$trainingstrategy$training_pct ) ]


if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

dtrain <- lgb.Dataset(
  data= data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
  label= dataset[fold_train == TRUE, clase01],
  free_raw_data= TRUE
)

In [ ]:
# datos de validation
dvalidate <- lgb.Dataset(
  data= data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$validate, campos_buenos, with = FALSE]),
  label= dataset[foto_mes %in% PARAM$trainingstrategy$validate, clase01],
  free_raw_data= TRUE
)

nrow(dvalidate)

#### 6.3.2.2. Hiperparámetros (Sin Bayesian Optimization)

**NO corremos BO.** Usamos directamente los mejores hiperparámetros encontrados en WF6300:
- num_leaves: 175
- min_data_in_leaf: 304
- num_iterations: 528

Los hiperparámetros ya fueron optimizados en WF6300. Saltamos directamente a la definición de parámetros fijos.

(Celda de BO omitida - usando hiperparámetros fijos)

(Celda de BO omitida)

(Celda de BO omitida)

(Celda de BO omitida)

In [ ]:
# NO se necesitan paquetes de BO - usamos hiperparámetros fijos
cat("Saltando Bayesian Optimization - usando hiperparámetros fijos de WF6300\n")

Definición de parámetros fijos de LightGBM (mismos que WF6300)

In [ ]:
# Parámetros fijos del LightGBM (mismos que WF6300)
PARAM$lgbm$param_fijos <- list(
  objective = "binary",
  metric = "auc",
  first_metric_only = TRUE,
  boost_from_average = TRUE,
  feature_pre_filter = FALSE,
  verbosity = -100,
  force_row_wise = TRUE,
  seed = PARAM$semilla_primigenia,
  max_bin = 31,
  learning_rate = 0.03,
  feature_fraction = 0.5
)

cat("Parámetros fijos definidos\n")

(Celda de función BO omitida)

In [ ]:
# Celda vacía - no se necesita función de BO
cat("Usando hiperparámetros fijos, no se necesita función de optimización\n")

(Celda de setup BO omitida)

In [ ]:
# Celda vacía - no se necesita setup de BO
cat("No se corre Bayesian Optimization\n")

(Celda de corrida BO omitida - saltamos directo a producción)

In [ ]:
# NO corremos BO - usamos los hiperparámetros ya encontrados
cat("Saltando Bayesian Optimization...\n")
cat("Hiperparámetros WF6300: num_leaves=175, min_data_in_leaf=304, num_iterations=528\n")

Asignamos directamente los mejores hiperparámetros de WF6300

In [ ]:
# Asignamos los mejores hiperparámetros de WF6300 directamente
# (ya los definimos al inicio en PARAM$lgbm$mejores_hiperparametros)

PARAM$out$lgbm$mejores_hiperparametros <- PARAM$lgbm$mejores_hiperparametros

cat("Mejores hiperparámetros WF6300:\n")
print(PARAM$out$lgbm$mejores_hiperparametros)

### 6.3.3 Produccion

#### Final Training
Construyo el modelo final, que es uno solo, no hace ningun tipo de particion < training, validation, testing>]

##### Final Training Dataset

Aqui esta la gran decision de en qué meses hago el Final Training
<br> debo utilizar los mejores hiperparámetros que encontré en la optimización bayesiana

In [ ]:
# Final Training Dataset - Mismos meses que WF6300
PARAM$trainingstrategy$final_train <- c(
  202107, 202106, 202105, 202104, 202103, 202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007,
  202006, 202005
)

dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train]

# creo el dfinal_train en formato LightGBM
dfinal_train <- lgb.Dataset(
  data = data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with = FALSE]),
  label = dataset[fold_final_train == TRUE, clase01],
  free_raw_data = TRUE
)

cat("Filas en final_train:", nrow(dfinal_train), "\n")
cat("Meses incluidos:", length(PARAM$trainingstrategy$final_train), "\n")

##### Final Training Hyperparameters

In [ ]:
# Combinar parámetros fijos con los mejores hiperparámetros de WF6300
param_final <- c(PARAM$lgbm$param_fijos, PARAM$out$lgbm$mejores_hiperparametros)

cat("Parámetros finales para el modelo:\n")
print(param_final)

##### Training
Genero el modelo final, siempre sobre TODOS los datos de  final_train, sin hacer ningun tipo de undersampling de la clase mayoritaria

In [ ]:
final_model <- lgb.train(
  data= dfinal_train,
  param= param_final,
  verbose= -100
)

In [ ]:
# grabo a disco el modelo en un formato para seres humanos ... ponele ...

lgb.save(final_model, "modelo.txt")

In [ ]:
# ahora imprimo la importancia de variables

tb_importancia <- as.data.table(lgb.importance(final_model))
archivo_importancia <- "impo.txt"

fwrite( tb_importancia,
  file= archivo_importancia,
  sep= "\t"
)

#### Scoring

Aplico el modelo final a los datos del futuro

In [ ]:
PARAM$trainingstrategy$future <- c(202107)

dfuture <- dataset[ foto_mes %in% PARAM$trainingstrategy$future ]

In [ ]:
# aplico final_model   a dfuture

prediccion <- predict(
  final_model,
  data.matrix(dfuture[, campos_buenos, with= FALSE])
)

##### Tabla Prediccion

In [ ]:
tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := prediccion]

# grabo las probabilidad del modelo
#  me va a ser util para hacer Ensembles de modelos
fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

#### Curva de Ganancia

Genero las salidas

In [ ]:
# asigno clase ternaria
tb_prediccion[, clase_ternaria := dfuture$clase_ternaria ]

# asigno ganancias
tb_prediccion[, ganancia := -3000.0 ]
tb_prediccion[clase_ternaria=="BAJA+2", ganancia := 117000.0 ]

# reordeno, y acumulada
setorder( tb_prediccion, -prob )
tb_prediccion[, gan_acum := cumsum(ganancia)]

# media movil de ancho 400
tb_prediccion[,
  gan_suavizada := frollmean(
    x= gan_acum,
    n= 400,
    align= "center",
    na.rm= TRUE,
    hasNA= TRUE
  )
]


In [ ]:
tb_prediccion[, sum(!is.na(gan_suavizada) )]
tb_prediccion[, .N]

In [ ]:
# Este es la ganancia a reportar
resultado <- list()
resultado$ganancia_suavizada_max <- max(tb_prediccion$gan_suavizada, na.rm = TRUE)
options(digits = 8)
resultado$envios <- which.max(tb_prediccion$gan_suavizada)

cat("\n========================================\n")
cat("RESULTADO - TEST JULIO 2021 (LOCAL)\n")
cat("========================================\n")
cat("Ganancia suavizada máxima:", resultado$ganancia_suavizada_max, "\n")
cat("Envíos óptimos:", resultado$envios, "\n")
cat("\n----------------------------------------\n")
cat("COMPARACIÓN CON KAGGLE (Sept 2021):\n")
cat("Kaggle WF6300 corte=1000: 4,587,000\n")
cat("========================================\n")

resultado

In [ ]:
# grabo las ganancias
fwrite( tb_prediccion,
  file= "ganancias.txt",
  sep= "\t"
)

In [ ]:
# genero el grafico

tb_prediccion[, envios:= .I]

pdf("curva_de_ganancia.pdf")

plot(
  x= tb_prediccion$envios,
  y= tb_prediccion$gan_acum,
  type= "l",
  col= "gray",
  xlim= c(0, 6000),
  ylim= c(0, 8000000),
  main= paste0("Mejor gan prom= ", as.integer(resultado$ganancia_suavizada_max), " envios= ", resultado$envios),
  xlab= "Envios",
  ylab= "Ganancia",
  panel.first= grid()
)

dev.off()

In [ ]:
# grabo los parametros
if( !require("yaml")) install.packages("yaml")
require("yaml")

PARAM$resultado <- resultado

write_yaml( PARAM, file="PARAM.yml")

In [ ]:
# Análisis de ganancia por corte - comparación con Kaggle
cortes_kaggle <- c(800, 850, 900, 950, 1000, 1050, 1100, 1150, 1200, 1250, 1300)

cat("\n========================================\n")
cat("GANANCIA POR CORTE - JULIO LOCAL\n")
cat("========================================\n")
cat(sprintf("%-8s %15s\n", "Corte", "Ganancia Julio"))
cat("----------------------------------------\n")
for (corte in cortes_kaggle) {
  if (corte <= nrow(tb_prediccion)) {
    gan_corte <- tb_prediccion[corte, gan_acum]
    cat(sprintf("%-8d %15.0f\n", corte, gan_corte))
  }
}
cat("========================================\n")
cat("\nKaggle WF6300 con corte=1000: 4,587,000\n")
cat("La diferencia muestra el drift temporal Jul->Sept\n")

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")